In [ ]:
import random
import pandas as pd
import hashlib
import os
from datetime import datetime as dt
import data_models as dm
import portfolio_strategy as strategy
import tc_optimization as tc
import data_analysis as analysis
import data.data_collection as dc
import indicators as indicator
import signalling

#reload modules
import importlib

importlib.reload(dm)
importlib.reload(strategy)
importlib.reload(tc)
importlib.reload(analysis)
importlib.reload(dc)
importlib.reload(indicator)
importlib.reload(signalling)


# Caminho do CSV
csv_path = "random_search_backtest_results_2.csv"

# Carrega resultados existentes (se houver)
if os.path.exists(csv_path):
    df_results = pd.read_csv(csv_path)
else:
    df_results = pd.DataFrame()

# Número de iterações
n_iterations = 10
results = []

for i in range(n_iterations):
    # Randomly generate parameters
    distance_method = random.choice(list(tc.DistanceMethod))
    #momentum_type = random.choice([dm.Momentum_Type.MACD, dm.Momentum_Type.Cumulative_Returns])
    momentum_type = random.choice([dm.Momentum_Type.MACD])
    momentum_threshold = round(random.uniform(1, 1.1), 6) # just for cumulative returns
    gamma = round(random.uniform(0, 3), 6)
    hurst_exponent_period = random.randint(180, 365)
    hurst_filter = random.choice(list(dm.HurstFilter))
    momentum_days = random.randint(5, 15) # just for cumulative returns

    # Create a unique strategy ID
    param_string = f"{distance_method}-{momentum_type}-{momentum_threshold}-{gamma}-{hurst_exponent_period}"
    strategy_id = hashlib.md5(param_string.encode()).hexdigest()

    # Verify if the strategy ID already exists in the results
    if strategy_id in df_results.get("Strategy_ID", []):
        continue

    Strategy = strategy.Strategy_Portfolio(
        start_date=dt(2022, 1, 2),
        end_date=dt(2024, 12, 31),
        Asset_Category=dm.Asset_Category.Top20CryptoByMarketCap,
        Momentum_Type=momentum_type,
        Mean_Rev_Type=dm.Mean_Rev_Type.RSI,
        Rebalancing_Period=dm.Rebalancing_Period.daily,
        Functional_Constraints=dm.Functional_Constraints(
            Take_Profit=0.2,
            Stop_Loss=0.1,
            Capital_at_Risk=0.6,
            Hurst_Filter=hurst_filter,
            RSIFilter=dm.RSIFilter.STANDARD,
            Hurst_Exponents_Period=hurst_exponent_period,
            Momentum_Threshold=momentum_threshold,
            MACD_Short_Window=12,
            MACD_Long_Window=26,
            Bollinger_Window=20
        ),
        Rebalance_Constraints=dm.Rebalance_Constraints(
            Long_Only=True,
            Turnover_Constraint=0.5,
            distance_method=distance_method,
            Transaction_Cost=0.01,
            Gamma=gamma,
            Delta_Range=(0, 1), #aumentar range?
            delta_step=1 
        ),
        Momentum_Days=momentum_days,
    )

    Strategy.run_strategy()
    performance = Strategy.backtest()
    stress = Strategy.stress_test()

    fc = Strategy.functional_constraints
    rc = Strategy.rebalance_constraints

    result_entry = {
        "Strategy_ID": strategy_id,
        "Iteration": i,
        "Start_Date": Strategy.start_date,
        "Asset_Category": str(Strategy.asset_category),
        "Momentum_Type": str(Strategy.momentum_type),
        "Mean_Rev_Type": str(Strategy.mean_rev_type),
        "Rebalancing_Period": str(Strategy.rebalancing_period),
        "Momentum_Days": Strategy.momentum_days,
        
        # Functional Constraints
        "Hurst_Filter": str(fc.hurst_filter),
        "RSIFilter": str(fc.rsi_filter),
        "Hurst_Exponents_Period": fc.hurst_exponents_period,
        "Momentum_Threshold": fc.momentum_threshold,
        "MACD_Short_Window": fc.macd_short_window,
        "MACD_Long_Window": fc.macd_long_window,
        "Bollinger_Window": fc.bollinger_window,
        
        # Rebalance Constraints
        "Long_Only": rc.long_only,
        "Distance_Method": str(rc.distance_method),
        "Transaction_Cost": rc.transaction_cost,
        "Gamma": rc.gamma,
        "Delta_Range_Start": rc.delta_range[0],
        "Delta_Range_End": rc.delta_range[1],
        "Delta_Step": rc.delta_step,
    }

    for metric_name, metric_value in performance.items():
        result_entry[metric_name] = metric_value

    results.append(result_entry)


IM RUNNING
True
Asset_Category.Top20CryptoByMarketCap
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan
Error to process: nan


/Users/guilhermegayoso/Desktop/TIC/Hurst/TICstrategy_PO/src/data/data_collection.py:124: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return close_prices.dropna(axis=1, thresh=len(close_prices) - 2).fillna(
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BTC-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


range(313, 1210)


In [ ]:
# Junta os novos resultados aos antigos e remove duplicatas
if result_entry:
    df_new = pd.DataFrame(results)
    df_final = pd.concat([df_results, df_new], ignore_index=True)
    df_final.drop_duplicates(subset=["Strategy_ID"], keep="last", inplace=True)
    df_final.to_csv(csv_path, index=False)
